# Compare Structure Function Calculations (X, Maya)

## Data = EMApex, small array:  EMApex_data_small_array_19-Feb-2025.mat

## 

In [14]:
# imports
from importlib import reload
import os, glob

import pymatreader

import seaborn as sns
from matplotlib import pyplot as plt

import numpy as np

import pandas

from profiler.specific import em_apex
from profiler import binning
from profiler import profilepairs

# Load me

In [3]:
dfile = '/home/xavier/Projects/Oceanography/data/ARCTERX/Floats/EM_Apex/EMApex_data_small_array_19-Feb-2025.mat'

d = pymatreader.read_mat(dfile)

In [4]:
d['E'].keys()

dict_keys(['F10285', 'F10280', 'F10282', 'F10281', 'F10278', 'F10279', 'F10283'])

# Check our Binning

In [7]:
dataset = 'ARCTERX-Leg2'
pDatas = em_apex.load_emapex_infield(dfile, dataset)

Working on float F10278
Working on float F10279
Working on float F10280
Working on float F10281
Working on float F10282
Working on float F10283
Working on float F10285


In [33]:
# Bin
#print(pDatas[0])
reload(binning)
bData = binning.bin_profilerdata(pDatas[0], add_vel=False)

FIX THIS HACK!!!!!!!!!!!!!!!!!!!!!!!!!
Depths: [ 10.  20.  30.  40.  50.  60.  70.  80.  90. 100. 110. 120. 130. 140.
 150. 160. 170. 180. 190. 200.]


## Check

In [34]:
bData

EMApexData object for ARCTERX-Leg2
  Number of profiles: 388
  Time range: 2025-02-05 23:18:34.045459509 to 2025-02-18 18:47:28.124996185
  Variables:
    time: (388,)
    lat: (388,)
    lon: (388,)
    t: (388, 20)
    s: (388, 20)
    theta: (388, 20)
    sigma: (388, 20)
    rho: (388, 20)

In [36]:
bData.depth

array([ 10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90., 100., 110.,
       120., 130., 140., 150., 160., 170., 180., 190., 200.])

### pandas Table for ease of reading

In [38]:
df = pandas.DataFrame(dict(depth=bData.depth, t=bData.t[0]))
df

,depth,t
0,10.0,24.872925
1,20.0,24.877049
2,30.0,24.877113
3,40.0,24.881218
4,50.0,24.886950
5,60.0,24.891440
6,70.0,24.888362
7,80.0,24.867585
8,90.0,24.858897
9,100.0,24.853914


## These match Maya's!!!!!!!

In [65]:
bData.time.size

388

In [67]:
dtimes = pandas.to_datetime(bData.time, unit='s')
dtimes[0]

Timestamp('2025-02-05 23:18:34.045459509')

----

# Structure Function

## Load all of the floats

In [11]:
reload(em_apex)
dataset = 'ARCTERX-Leg2'
dfile = '/home/xavier/Projects/Oceanography/data/ARCTERX/Floats/EM_Apex/EMApex_data_small_array_19-Feb-2025.mat'
emapexs = em_apex.load_emapex_infield(dfile, dataset, add_vel=False,
                                     skip_floats=['F10285'])

Working on float F10278
FIX THIS HACK!!!!!!!!!!!!!!!!!!!!!!!!!
Working on float F10279
FIX THIS HACK!!!!!!!!!!!!!!!!!!!!!!!!!
Working on float F10280
FIX THIS HACK!!!!!!!!!!!!!!!!!!!!!!!!!
Working on float F10281
FIX THIS HACK!!!!!!!!!!!!!!!!!!!!!!!!!
Working on float F10282
FIX THIS HACK!!!!!!!!!!!!!!!!!!!!!!!!!
Working on float F10283
FIX THIS HACK!!!!!!!!!!!!!!!!!!!!!!!!!
Skipping F10285


In [7]:
variables='dTdTdT'

## Radial bins

In [8]:
nbins = 20
rbins = 10**np.linspace(0., np.log10(400), nbins) # km

## Pairs

In [66]:
reload(profilepairs)
pPairs = profilepairs.ProfilerPairs(
            emapexs, max_time=10., 
            avoid_same_glider=True, debug=False)

Using lonendpts: (np.float64(129.9118847499992), np.float64(130.48001875000182))
Using latendpts: (np.float64(20.487747250001583), np.float64(22.94666474999614))


In [67]:
pPairs

ProfilerPair object for the following datasets:
 ARCTERX-Leg2, EMApexData
 ARCTERX-Leg2, EMApexData
 ARCTERX-Leg2, EMApexData
 ARCTERX-Leg2, EMApexData
 ARCTERX-Leg2, EMApexData
 ARCTERX-Leg2, EMApexData
  Number of pairs: 145756
  Time range: 2025-02-05 23:18:34.045459509 to 2025-02-18 19:52:11.236361980

## The number of profiles are within 6 of Maya

## We declare victory because our times aren't identical

## Do it

In [68]:
emapexs[0].depth[5]

np.float64(60.0)

In [69]:
rbins

array([  1.        ,   1.37072561,   1.87888869,   2.57544084,
         3.5302227 ,   4.83896665,   6.6328955 ,   9.09187971,
        12.46247232,  17.08262993,  23.41559827,  32.09636014,
        43.99530272,  60.305488  ,  82.6622766 , 113.30729922,
       155.31321644, 212.89180278, 291.81624545, 400.        ])

In [70]:
iz = 5

pPairs.calc_delta(iz, variables, skip_velocity=True)
pPairs.calc_Sn(variables)

Sn_dict = pPairs.calc_Sn_vs_r(rbins, nboot=100)
#pPairs.calc_corr_Sn(Sn_dict) 
pPairs.add_meta(Sn_dict)

/home/xavier/Projects/Oceanography/python/profiler/profiler/profilepairs.py:493: RuntimeWarning: Mean of empty slice
  out_dict['std_S2_'+f'{self.dlbls[1]}'] = np.array(std_S2)
/home/xavier/Projects/Oceanography/python/profiler/profiler/profilepairs.py:494: RuntimeWarning: Mean of empty slice
  out_dict['err_S2_'+f'{self.dlbls[1]}'] = np.array(err_S2)
/home/xavier/miniconda3/envs/ocean/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1231: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/home/xavier/miniconda3/envs/ocean/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2035: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/xavier/Projects/Oceanography/python/profiler/profiler/profilepairs.py:497: RuntimeWarning: Mean of empty slice
  out_dict['err_S3_'+f'{self.dlbls[2]}'] = np.array(err_S3)
/home/xavier/Projects/Oceanography/python/profiler/profiler

## The big reveal!

In [71]:
Sn_dict.keys()

dict_keys(['config', 'r', 'S1_dT', 'std_S1_dT', 'med_S1_dT', 'err_S1_dT', 'S2_dT**2', 'std_S2_dT**2', 'err_S2_dT**2', 'S3_dTdTdT', 'std_S3_dTdTdT', 'err_S3_dTdTdT'])

In [72]:
Sn_dict['config']['N']

array([ 3509,  5455,  7739, 10753, 14690, 19048, 21958, 21250, 18292,
       11256,  6609,   533,     0,     0,     0,     0,     0,     0,
           0])

In [73]:
Sn_dict['r']

array([ 1.18745972,  1.6294386 ,  2.22820461,  3.052704  ,  4.18119434,
        5.7245081 ,  7.80645494, 10.65312137, 14.49974422, 19.92495186,
       26.84336271, 33.99738493,         nan,         nan,         nan,
               nan,         nan,         nan,         nan])

In [74]:
Sn_dict['S1_dT']

array([ 4.16937257e-04,  2.49248223e-04,  2.52663166e-04, -3.99464598e-04,
        6.08047306e-04,  1.61780111e-03, -1.87710558e-03,  1.24082386e-06,
        1.96118245e-03, -8.35299350e-04,  1.62308763e-03, -5.59236063e-03,
                   nan,             nan,             nan,             nan,
                   nan,             nan,             nan])

----

In [45]:
a = np.ones((100,20))
b = np.ones((80,20))
#
c = np.concatenate([a,b],axis=0)
c.shape

(180, 20)